## 1. Implement the uncollapsed Gibbs sampler for latent Dirichlet allocation we dis- cussed in class. Apply it to state-of-the-union addresses at a level of aggregation you choose and describe the output.

First, we need to set up the environment and take care of all the boring stuff (we will recycle code from the previous exercise to handle the text data)

In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk import PorterStemmer, word_tokenize
from numpy.random import dirichlet
from collections import Counter
from scipy.sparse import csr_matrix
import time

data = pd.read_table(path+"speech_data_extend.txt",encoding="utf-8")
data = data.loc[data['year']>=1946]
data = data.reset_index()

def data_preparation(data):
    prep_data = data.apply(lambda row: #tokenize
                            word_tokenize(row['speech'].lower()), axis=1)
    stop_w=set(stopwords.words('english')) #stopwords
    for i in range(len(prep_data)): #non-alphanumeric characters
        prep_data[i] = [w for w in prep_data[i] if w not in stop_w and w.isalpha()]
    stemmer = PorterStemmer() #Create a stemmer object
    for i in range(len(prep_data)): #Stem the data
        prep_data[i] = [stemmer.stem(elem) for elem in prep_data[i]]
    unique_words = np.unique([word for doc in prep_data for word in doc]).tolist() #List of unique words
    D = len(prep_data)
    V = len(unique_words)
    X = np.zeros((D,V)) #The document-term matrix
    N = 0
    for i in range(D):
        N = N + len(prep_data[i])
        aux_words_d = list(set(prep_data[i]))
        for j in range(len(aux_words_d)):
            X[i,unique_words.index(aux_words_d[j])] = prep_data[i].count(aux_words_d[j])
    X = csr_matrix(X.astype(int))
    return prep_data, unique_words, X, N

prep_data, unique_words, X, N = data_preparation(data)

### Sample topic allocation
First, we create a function that simulates from a multinomial and actually returns draws from a multinomial. That way, we can generate an initial guess for the Z 'matrix'.

In [ ]:
def simulate(K, row):
    samples = np.random.multinomial(1,[1/K]*K,len(prep_data[row])).tolist()
    samples_correct = []
    for s in samples:
        samples_correct.append(s.index(1))
    return samples_correct

Z = prep_data.apply(lambda row: simulate(K,row))

Note that Z is not a matrix, but a list of sublists. There are D sublists (one for each document), each one containing n_d entries (different documents have different number of words).
Now, we create a function that samples from those topics

In [ ]:
def sample_topic(Z, theta, beta):
    D = len(Z)
    for d in range(D):
        n = len(Z[d])
        for i in range(n):
            beta_v = beta[unique_words.index(prep_data[d][i])]
            probs = (theta[d,:]*beta_v)/np.sum(theta[d,:]*beta_v)
            Z[d][i] = np.random.multinomial(1, probs).tolist().index(1)
    return Z

### Sample theta

For the theta, we'll need two functions: (i) a function that generates the number of counts per document and topic, and (ii) the function that actually samples from theta.

In [ ]:
def N_count(Z_d, K):
    N_count_vector = []
    for k in range(K):
        N_count_vector.append(Z_d.count(k))
    return N_count_vector

def sample_theta(Z,alpha,theta):
    D,K = theta.shape
    N = np.zeros((D,K))
    for d in range(D):
        N[d,:] = N_count(Z[d], K)
        theta[d,:] = dirichlet(N[d,:] + alpha)
    return theta


### Sample beta

Now, we will create a function that generates the betas. Note that it includes the script to generate the M, which it is needed to generate the betas.

In [ ]:
def sample_beta(Z,prep_data,eta,beta):
    K = beta.shape[1]
    M = np.zeros((K,V))
    #Generate M
    s = [i for sublist in prep_data for i in sublist ]
    z_s = [z for sublist in Z for z in sublist]
    for k in range(K):
        words = [s[i] for i in range(len(s)) if z_s[i] == k]
        counts = Counter(words)
        for v in range(len(unique_words)):
            if unique_words[v] in counts: M[k,v] = counts[unique_words[v]]
    #Generate beta
    for k in range(K):
        beta[:,k] = dirichlet(M[k,:] + eta)
    return beta

### The sampler

Finally, we put it all together inside a function, iterate and compute the perplexity

In [ ]:
def gibbs_sampler(n_iter,prep_data,alpha,eta, K, X, N, prop_perplexity):
    ## Initialize objects
    D = len(prep_data)
    theta = dirichlet([alpha]*K,D)
    beta = dirichlet([eta]*K,V)
    Z = prep_data.apply(lambda row: simulate(K,row))
    Z_dist = []
    theta_dist = []
    beta_dist = []
    perplexity = []
    for i in range(n_iter):
        print('Iteration nº:'+ str(i))
        start = time.time()
        Z = sample_topic(Z,theta,beta)
        theta = sample_theta(Z,alpha,theta)
        beta = sample_beta(Z,prep_data,eta,beta)
        if (i % (round(n_iter * prop_perplexity) + 1)) == 0:
            perplexity.append(np.exp(-np.sum(X.multiply(np.log(theta.dot(beta.T))))/N))
            np.save(path2+"perplexity.npy",perplexity)
        Z_dist.append(Z)
        theta_dist.append(theta)
        beta_dist.append(beta)
        np.save(path2+"theta.npy",theta_dist)
        np.save(path2+"Z_dist.npy",Z_dist)
        np.save(path2+"beta_dist.npy",beta_dist)
        print('Duration:'+ str(time.time()-start))
    return Z_dist, beta_dist, theta_dist, perplexity


#Initial values (reference original paper)
K = 2 #Number of topics
alpha = 50/K
V = len(unique_words)
eta = 200/V

We've tried with the parameters indicated above. Unfortunately, each iteration took 30 seconds, so we could not run a proper round. Trying with different parameters, we saw that for higher K, perplexity is lower. But it runs, so you can check it works (maybe try low number of iterations, it will show the progress in any case).

## 3. Now focus on individual paragraphs of state-of-the-union addresses from 1946 on- wards in which every paragraph is associated with one of two political parties.

Again, let's set up the environment and load everything that is needed.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Select only documents which appear after 1945
X = count_words(prep_data)
# Create a new variable with only presidents and years
parties = data.loc[:,['president', 'year']]
parties = parties.reset_index()
# Create a new variable with 1 when presidents are Democrats, 0 when they are Republicans
zero_len = pd.Series(np.zeros(len(parties.index)))
parties['parties'] = zero_len
parties['parties'] = (parties.president == "Truman") | (parties.president == "Kennedy") | (parties.president == "Johnson") | (parties.president == "Carter") | (parties.president == "Clinton") | (parties.president == "Obama")
parties.parties = list(map(lambda x: 1 if x else 0, parties.parties))
parties = parties.iloc[:][parties.year >= 1945]
# Set the parties variable as y
y = parties.parties
# Split the X and y variables into a training and test set
X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y, test_size = 0.3, random_state = 42)

NameError: name 'count_words' is not defined

Now, let's start with the fun stuff. First, we first a logistic regression model and do some basic parameter tuning.

In [2]:
# Create the logistic regression estimator with an l1 loss parameter
log_reg = LogisticRegression(penalty = "l1")
# Set some parameters to tune over
c_space = [1.3, 1.5, 1.7]
parameters = {'C': c_space}
# Create a cross-validation estimator
cv = GridSearchCV(log_reg, parameters)
# Fit the model, predict and report the accuracy and the best parameter
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
print("Accuracy: {}".format(cv.score(X_test, y_test)))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

NameError: name 'X_train' is not defined

Now, we'll try with a logistic regression using the topics instead of the document-term matrix

In [ ]:
X = count_words(prep_data)
K, S, alpha, eta = 2, 1000, 0.1, 0.01
Col_Gibbs = lda.LDA(n_topics=K, n_iter=S, alpha=alpha, eta=eta)
X = Col_Gibbs.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 59)
log_reg = LogisticRegression("l2")
c_space = [0.5, 2, 5]
parameters = {'C': c_space}
# Create a cross-validation estimator
cv = GridSearchCV(log_reg, parameters)
# Fit the model, predict and report the accuracy and the best parameter
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
print("Accuracy: {}".format(cv.score(X_test, y_test)))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))